In [1]:
# database

import os
import shutil
import pandas as pd
import xml.etree.ElementTree as XET

import sqlalchemy as sqlc
from sqlalchemy import exc

from sqlalchemy import Table, Column, MetaData, Integer, Computed, String
from synology_dsm import SynologyDSM
from synology_api import filestation

from configparser import ConfigParser, ExtendedInterpolation
from time import strftime
from tqdm import tqdm

config = ConfigParser(interpolation=ExtendedInterpolation())
config.read('./config/db_config.ini', encoding='utf-8-sig')

db_info = config['database']
host_ip = db_info['host_ip']
port = db_info['port']
user_name = db_info['user_name']
password = db_info['password']
db_name = db_info['db_name']
db_url = db_info['db_url']

dataset_path = config['dataset']['dir_path']
saved_path = config['dataset']['saved_path']

In [2]:
class SQL_Config():
    """
    SQL SERVER 連接設定
    """
    def __init__(self, user=None, password=None, database=None, host_address=None, db_url=None, port=32769):
        self.user = user
        self.password = password
        self.db_name = database
        self.host = host_address
        self.created_time = strftime('%Y-%m-%d %H:%M:%S')
        self.port = port
        
        if db_url is None:
            self.db_url = f"{user}:{password}@{host_address}:{port}/{database}"
        else:
            self.db_url = db_url

    def postgreSQL_connect(self):
        """
        進行 SQL SERVER 連接
        """
        print('=====================================================')
        print('======== Connect to the remote postgre SQL server ========')
        print('=====================================================')
        print('Time : {}\n'.format(strftime('%Y-%m-%d_%H_%M')))
        
        sql_prefix = "postgresql+psycopg2://" + self.db_url
            
        self.engine = sqlc.create_engine(sql_prefix)
        self.db = self.engine.raw_connection()
        self.cursor = self.db.cursor()

    def commit(self):
        """
        資料庫commit
        """
        try:
            trans = self.db
            trans.commit()
        except:
            trans.rollback()

    def disconnect(self):
        """
        資料庫離線
        """
        self.db.close()
        print('=====================================================')
        print('============ Close the remote connection ============')
        print('=====================================================')

In [3]:
class Table_Creator(SQL_Config):
    """
    Create SQL Table with sqlalchemy
    """
    def __init__(self, user=None, password=None, database=None, host_address=None, db_url=None, port=32769):
        super().__init__(user, password, database, host_address, db_url, port)

    def create_table4annot(self):
        metadata = MetaData()

        data = Table(
            "Image_Annot", metadata,
            Column('UUID', Integer, autoincrement=True, primary_key=True),
            Column('Name', String),
            Column('Pose', String),
            Column('Truncated', Integer),
            Column('Difficult', Integer),
            Column('Xmin', Integer),
            Column('Ymin', Integer),
            Column('Xmax', Integer),
            Column('Ymax', Integer),
            Column('File', String),
            Column('Dataset', String)
            )
        metadata.create_all(self.engine)

    def create_table4label(self):
        metadata = MetaData()

        data = Table(
            "Image_Label", metadata,
            Column('Label_ID', Integer),
            Column('Name', String, primary_key=True),
            Column('Dataset', String, primary_key=True)
            )

        metadata.create_all(self.engine)
        
    def create_table4image(self):
        metadata = MetaData()

        data = Table(
            "Image_Info", metadata,
            Column('File_Name', String, primary_key=True),
            Column('Width', Integer),
            Column('Height', Integer),
            Column('Depth', Integer),
            Column('Dataset', String, primary_key=True)
            )

        metadata.create_all(self.engine)
        
    def create_table4dataset(self):
        metadata = MetaData()

        data = Table(
            "Dataset", metadata,
            Column('Name', String),
            Column('Prefix_Path', String, primary_key=True),
            )

        metadata.create_all(self.engine)
        
    def start(self):
        self.postgreSQL_connect()
        
        self.create_table4annot()
        self.create_table4label()
        self.create_table4image()
        self.create_table4dataset()

In [4]:
Table_Creator(user=user_name, password=password, database=db_name, host_address=host_ip, port=port).start()

======== Connect to the remote postgre SQL server ========
Time : 2020-12-23_23_01



In [5]:
class NAS_Config():
    """
    Synology DSM 連接設定
    """
    def __init__(self, user=None, password=None, host_address=None, port="5000"):
        self.user = user
        self.password = password
        self.host = host_address
        self.created_time = strftime('%Y-%m-%d %H:%M:%S')
        self.port = port
        
        self.connect()

    def connect(self):
        """
        連接 Synology NAS 
        """
        print('=====================================================')
        print('======== Connect to the remote NAS server ========')
        print('=====================================================')
        print('Time : {}\n'.format(strftime('%Y-%m-%d_%H_%M')))
            
        self.api = SynologyDSM(self.host, self.port, self.user, self.password)

    def print_system_info(self):
        """
        顯示系統資訊
        """
        print("=== Information ===")
        self.api.information.update()
        print("Model:           " + str(self.api.information.model))
        print("RAM:             " + str(self.api.information.ram) + " MB")
        print("Temperature:     " + str(self.api.information.temperature) + " °C")
        print("--")

    def print_utilisation(self):
        """
        顯示資源使用率
        """
        self.api.utilisation.update()
        print("CPU Load:        " + str(self.api.utilisation.cpu_total_load) + " %")
        print("Memory Use:      " + str(self.api.utilisation.memory_real_usage) + " %")
        print("Net Up:          " + str(self.api.utilisation.network_up()))
        print("Net Down:        " + str(self.api.utilisation.network_down()))
        print("--")

    def print_storage_info(self):  
        """
        顯示儲存裝置資訊
        """
        print("=== Storage ===")
        self.api.storage.update()
        for volume_id in self.api.storage.volumes_ids:
            print("ID:          " + str(volume_id))
            print("Status:      " + str(self.api.storage.volume_status(volume_id)))
            print("% Used:      " + str(self.api.storage.volume_percentage_used(volume_id)) + " %")
            print("--")
            
        for disk_id in self.api.storage.disks_ids:
            print("ID:          " + str(disk_id))
            print("Name:        " + str(self.api.storage.disk_name(disk_id)))
            print("Status:      " + str(self.api.storage.disk_status(disk_id)))
            print("Temp:        " + str(self.api.storage.disk_temp(disk_id)))
            print("--")
            
    def print_shared_folder(self):  
        """
        顯示共用空間資訊
        """ 
        print("=== Shared Folders ===")
        self.api.share.update()
        for share_uuid in self.api.share.shares_uuids:
            print("Share name:        " + str(self.api.share.share_name(share_uuid)))
            print("Share path:        " + str(self.api.share.share_path(share_uuid)))
            print("Space used:        " + str(self.api.share.share_size(share_uuid, human_readable=True)))
            print("--")

In [6]:
class Filestation_Config():
    """
    Synology Filestation 連接設定
    """
    def __init__(self, user=None, password=None, host_address=None, port="5000"):
        self.user = user
        self.password = password
        self.host = host_address
        self.created_time = strftime('%Y-%m-%d %H:%M:%S')
        self.port = port
        
        self.connect()

    def connect(self):
        """
        連接 Synology NAS 
        """
        print('=====================================================')
        print('======== Connect to the remote filestation ========')
        print('=====================================================')
        print('Time : {}\n'.format(strftime('%Y-%m-%d_%H_%M')))
            
        self.api = filestation.FileStation(self.host, self.port, self.user, self.password)
            
    def print_shared_folder(self):  
        """
        顯示共用空間資訊
        """ 
        print("=== Shared Folders ===")
        query_info = self.api.get_list_share()
        if not query_info['success']:
            print('query shared folder failed...')
            
        for folder_info in query_info['data']['shares']:
            if folder_info['isdir']:
                print(folder_info['name'])
                print(folder_info['path'])
                print('---')
                
    def query_file_list(self, dir_path):  
        """
        顯示資料夾內容
        """ 
        query_info = self.api.get_file_list(dir_path)
        file_df = pd.DataFrame([], columns=["Name", "Path"])
        if not query_info['success']:
            print('query file list failed...')
                
        for folder_info in query_info['data']['shares']:
            print('data list')
            if not folder_info['isdir']:
                print('name: ', folder_info['name'])
                print('path: ', folder_info['path'])
                file_df = file_df.append({"File_Name": folder_info['name'], "File_Path": folder_info['path']}, 
                                         ignore_index=True)
                print('---')
                
        return file_df
    
    def query_dir_info(self, dir_path):  
        """
        顯示資料夾內容
        """ 
        query_info = self.api.get_file_list(dir_path)
        dir_dict = {}
        if not query_info['success']:
            print('query dir list failed...')
            
        for folder_info in query_info['data']['files']:
            print('dir info')
            if folder_info['isdir']:
                print('name: ', folder_info['name'])
                print('path: ', folder_info['path'])
                dir_dict[folder_info['name']] = folder_info['path']
                print('---')
                
        return dir_dict
    
    def download_file(self, path, file_name, dst_path):  
        if not os.path.exists(dst_path):
            os.makedirs(dst_path)
            
        file_path = path + '/' + file_name
        self.api.get_file(file_path, 'download')
        shutil.move(file_name, dst_path)

In [7]:
dsm_api = Filestation_Config(user_name, password, host_ip)
dsm_api.print_shared_folder()

======== Connect to the remote filestation ========
Time : 2020-12-23_23_01

You are now logged in!
=== Shared Folders ===
ActiveBackupforBusiness
/ActiveBackupforBusiness
---
chat
/chat
---
dataset
/dataset
---
docker
/docker
---
home
/home
---
homes
/homes
---
music
/music
---
nas
/nas
---
photo
/photo
---
video
/video
---
web
/web
---


In [8]:
class Image_Dataset2DB(SQL_Config):
    """
    Upload the public dataset information to SQL
    """
    def __init__(self, user=None, password=None, database=None, host_address=None, db_url=None, port=32769):
        super().__init__(user, password, database, host_address, db_url)
        self.connect2NAS()
        
    def connect2NAS(self):
        self.dsm_api = NAS_Config(self.user, self.password, self.host)
        self.filestation_api = Filestation_Config(self.user, self.password, self.host)
        
        print('connct to NAS...')    
        self.dsm_api.print_utilisation()
        self.dsm_api.print_system_info()

    def upload_label(self, label_df):
        self.upload_dataset2sql(label_df, "Image_Label")    

    def upload_BBoxes(self, annot_df):
        self.upload_dataset2sql(annot_df, "Image_Annot")  
        
    def upload_imageInfo(self, image_df):  
        self.upload_dataset2sql(image_df, "Image_Info")
        
    def upload_datasetInfo(self, dataset_df):
        self.upload_dataset2sql(dataset_df, "Dataset")      
            
    def upload_dataset2sql(self, df, table_name, if_exists='append'):
        for idx in range(len(df)):
            try:
                df.iloc[idx:idx+1].to_sql(table_name, self.engine, if_exists=if_exists, index=False)
            except exc.IntegrityError:
                pass

In [9]:
class VOC_Dataset(Image_Dataset2DB):
    """
    Extract information of VOC dataset
    """
    def __init__(self, dir_path, dataset_name, 
                 user=None, password=None, database=None, host_address=None, db_url=None, port=32769):
        self.dataset_name = dataset_name
        self.dir_path = dir_path
        
        super().__init__(user, password, database, host_address, db_url)
        
    def extract_dataset_info(self, nas_path):
        dataset_df = pd.DataFrame([], columns=["Name", "Prefix_Path"])
        dir_dict = self.filestation_api.query_dir_info(nas_path)
    
        for folder_name in dir_dict.keys():
            if self.dataset_name == folder_name:
                dataset_df = dataset_df.append({"Name": folder_name, "Prefix_Path": dir_dict[folder_name]},
                                           ignore_index=True)
                
        self.upload_datasetInfo(dataset_df)
    
    def extract_label_info(self):
        path = self.dir_path + 'Annotations/'
        bbox_df = pd.DataFrame([], columns=["Name", "Pose", "Truncated", "Difficult",
                                             "Xmin", "Ymin", "Xmax", "Ymax",
                                             "File", "Dataset"])
        image_df = pd.DataFrame([], columns=["File_Name", "Width", "Height", "Depth", "Dataset"])
    
        file_list = os.listdir(path)
    
        for file in file_list:
            tree = XET.parse(path + file)
            root = tree.getroot()
            
            dataset = root.findall('folder')[0].text
            assert(self.dataset_name == dataset)
            
            file_name = root.findall('filename')[0].text
            size = root.findall('size')[0]
        
            width = int(size[0].text)
            height = int(size[1].text)
            depth = int(size[2].text)
        
            image_df = image_df.append({"File_Name": file_name,
                                        "Width": width,
                                        "Height": height,
                                        "Depth": depth,
                                        "Dataset": self.dataset_name},
                                       ignore_index=True)
            bbox_df = self.extract_BBoxes(root, bbox_df, file_name)    
            
        self.upload_imageInfo(image_df)
        self.upload_BBoxes(bbox_df)
        
    def extract_BBoxes(self, root, bbox_df, file_name):
        objects = root.findall('object')
    
        for obj in objects:
            name = obj.find('name').text
            pose = obj.find('pose').text
            truncated = obj.find('truncated').text
            difficult = obj.find('difficult').text
        
            bndbox = obj.find('bndbox')
            xmin = int(bndbox[0].text)
            ymin = int(bndbox[1].text)
            xmax = int(bndbox[2].text)
            ymax = int(bndbox[3].text)
        
            bbox_df = bbox_df.append({"Name": name, 
                                      "Pose": pose, 
                                      "Truncated": truncated, 
                                      "Difficult": difficult, 
                                      "Xmin": xmin, 
                                      "Ymin": ymin, 
                                      "Xmax": xmax, 
                                      "Ymax": ymax, 
                                      "File": file_name, 
                                      "Dataset": self.dataset_name}, 
                                     ignore_index=True)
        return bbox_df
    
    def create_label_ID(self, label_path):
        label_df = pd.DataFrame([], columns=["Label_ID", "Name", "Dataset"])

        with open(label_path, "r") as f:
            lines = f.read().splitlines()
            for idx, line in enumerate(lines):
                label_df = label_df.append({"Label_ID": idx, "Name": line, "Dataset": self.dataset_name}, ignore_index=True)

            self.upload_dataset2sql(label_df, "Image_Label")
        
        label_dict = label_df.set_index('Name').to_dict()
        return label_dict["Label_ID"]

In [10]:
prefix = dataset_path.split('/')[-2]

voc = VOC_Dataset(dataset_path, prefix,
                  user=user_name, password=password, database=db_name, host_address=host_ip, port=port)

voc.postgreSQL_connect()
label_dict = voc.create_label_ID('./label/VOC2007.txt')
voc.extract_label_info()
voc.extract_dataset_info(nas_path='/dataset')

======== Connect to the remote NAS server ========
Time : 2020-12-23_23_01

======== Connect to the remote filestation ========
Time : 2020-12-23_23_01

You are now logged in!
connct to NAS...
CPU Load:        6 %
Memory Use:      46 %
Net Up:          3798
Net Down:        4921
--
=== Information ===
Model:           DS720+
RAM:             6144 MB
Temperature:     35 °C
--
======== Connect to the remote postgre SQL server ========
Time : 2020-12-23_23_01

dir info
name:  #recycle
path:  /dataset/#recycle
---
dir info
name:  VOC2007
path:  /dataset/VOC2007
---


In [11]:
class Dataset_Download(SQL_Config):
    """
    Download the dataset from SQL
    """
    def __init__(self, dataset_name, user=None, password=None, database=None, host_address=None, db_url=None, port=32769):
        super().__init__(user, password, database, host_address, db_url, port)
        self.filestation = Filestation_Config(user, password, host_address)
        
        self.dataset_name = dataset_name
        self.postgreSQL_connect()

    def download_label(self, sql_query=None):
        if sql_query is None:
            sql_query = 'SELECT * FROM "Image_Label" WHERE "Dataset"=\'' + self.dataset_name + '\';'
            
        file_name = self.dataset_name + '_label_data'
        label_df = self.download_sql_table(sql_query, file_name)
        return label_df

    def download_BBoxes(self, sql_query=None):
        if sql_query is None:
            sql_query = 'SELECT * FROM "Image_Annot" WHERE "Dataset"=\'' + self.dataset_name + '\';'
            
        file_name = self.dataset_name + '_bboxes_data'
        bboxes_df = self.download_sql_table(sql_query, file_name)
        return bboxes_df
        
    def download_imageInfo(self, sql_query=None):  
        if sql_query is None:
            sql_query = 'SELECT * FROM "Image_Info" WHERE "Dataset"=\'' + self.dataset_name + '\';'
            
        file_name = self.dataset_name + '_image_info'
        image_df = self.download_sql_table(sql_query, file_name)
        return image_df
    
    def download_image(self, dataset_df, image_df): 
        path = dataset_df.set_index('Name').loc['VOC2007', 'Prefix_Path']
        img_files = image_df.set_index('Dataset').loc['VOC2007', 'File_Name']
        dst_dir = saved_path + 'image/' + self.dataset_name
        print('Start download image...')
        
        for file_name in tqdm(img_files, ncols=60):
            self.filestation.download_file(path, file_name, dst_dir)
            
        print('All the images are downloaded')
        
    def download_datasetInfo(self, sql_query=None):
        if sql_query is None:
            sql_query = 'SELECT * FROM "Dataset" WHERE "Name"=\'' + self.dataset_name + '\';'
            
        file_name = self.dataset_name + '_dataset'
        dataset_df = self.download_sql_table(sql_query, file_name)
        return dataset_df
        
    def download_sql_table(self, sql_query, file_name):
        if not os.path.exists(saved_path):
            os.makedirs(saved_path)
            
        print('Start to query...')
        table_df = pd.read_sql(sql_query, self.db)
        table_df.to_csv(saved_path + file_name + '.csv', index=False, encoding='utf_8_sig')
        print('==> The ' + file_name + '.csv is saved \n')
            
        return table_df
        
    def start(self):
        label_df = self.download_label()
        bboxes_df = self.download_BBoxes()
        image_df = self.download_imageInfo()
        dataset_df = self.download_datasetInfo()
        
        self.download_image(dataset_df, image_df)

In [12]:
Dataset_Download('VOC2007', user=user_name, password=password, database=db_name, host_address=host_ip, port=port).start()

======== Connect to the remote filestation ========
Time : 2020-12-23_23_10

You are now logged in!
======== Connect to the remote postgre SQL server ========
Time : 2020-12-23_23_10

Start to query...
==> The VOC2007_label_data.csv is saved 

Start to query...


  0%|                              | 0/5011 [00:00<?, ?it/s]

==> The VOC2007_bboxes_data.csv is saved 

Start to query...
==> The VOC2007_image_info.csv is saved 

Start to query...
==> The VOC2007_dataset.csv is saved 

Start download image...


100%|███████████████████| 5011/5011 [11:25<00:00,  7.31it/s]

All the images are downloaded
